In [1]:
import pandas as pd
import numpy as np

print('Версия pandas:', pd.__version__)

Версия pandas: 2.1.4


# Сводные таблицы в pandas.
# Функция CROSSTAB


## Часть 2. Фрагмент 3.

    - Введение
    - Открываем файл handedness_all.csv
    1. Используем метод value_counts с отдельной колонкой (Series)
    2. Используем метод value_counts с таблицей (DataFrame)
    3. Используем функцию pd.crosstab
    4. Передаем одновременно три переменные (или более)
    5. Параметр normalize (index, columns, all (True))
    6. Параметры margins_name, rownames и colnames
    7. Параметры values и aggfunc в pd.crosstab
    8. pd.crosstab + Categorical. Параметр dropna

In [2]:
# открываем сохраненный файл с дополнительными параметрами index_col и header
df_hand=pd.read_csv('files/handedness', index_col=[0,1], header=[0,1])
df_hand

gender                         M           F      
handedness                  left right  left right
city          birth_season                        
Moscow        winter        1284  1167  1190  1360
              spring        1424  1135  1425   928
              summer         907   890  1029   787
              autumn        1010   961   890   896
St.Petersburg winter         919   991   912  1282
              spring        1001  1471  1282  1410
              summer        1343   934   777   939
              autumn        1402  1264  1425   961

In [3]:
# откроем файл с результатами опроса 
df_h=pd.read_csv('files/handedness_all')
# нет отсутвующих значений NaN
df_h.info()
df_h

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34995 entries, 0 to 34994
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            34995 non-null  int64 
 1   city          34995 non-null  object
 2   birth_season  34995 non-null  object
 3   handedness    34995 non-null  object
 4   gender        34995 non-null  object
dtypes: int64(1), object(4)
memory usage: 1.3+ MB


,ID,city,birth_season,handedness,gender
0,30952,St.Petersburg,autumn,left,M
1,26972,St.Petersburg,summer,left,M
2,17961,St.Petersburg,winter,left,M
3,21035,St.Petersburg,winter,right,M
4,13615,Moscow,autumn,left,M
...,...,...,...,...,...
34990,9639,Moscow,summer,left,M
34991,20869,St.Petersburg,winter,right,M
34992,29944,St.Petersburg,summer,right,F
34993,21803,St.Petersburg,winter,right,F


# 1. Используем метод value_counts с отдельной колонкой (Series)

## Пример № 1.1

In [4]:
print(df_h['gender'].value_counts(),
      df_h['gender'].value_counts(normalize=True).round(3)*100, sep='\n\n')

gender
F    18116
M    16879
Name: count, dtype: int64

gender
F    51.8
M    48.2
Name: proportion, dtype: float64


## Пример № 1.2

In [5]:
print(df_h['handedness'].value_counts(),
      df_h['handedness'].value_counts(normalize=True).round(3)*100, sep='\n\n')

handedness
right    17620
left     17375
Name: count, dtype: int64

handedness
right    50.4
left     49.6
Name: proportion, dtype: float64


# 2. Используем метод value_counts с таблицей (DataFrame)

## Пример № 2.1

In [6]:
# больше всего в наборе людей из категории 'женщины-правши из Москвы, родившиеся зимой' - 1486 человек
# меньше всего в наборе людей из категории 'мужчины-правши из Москвы, родившиеся зимой' - 816 человек

df_h.iloc[:,[1,2,3,4]].value_counts()

city           birth_season  handedness  gender
Moscow         winter        right       F         1486
                             left        M         1485
St.Petersburg  spring        right       F         1482
Moscow         summer        left        M         1365
               autumn        right       F         1330
St.Petersburg  winter        left        F         1306
                                         M         1257
Moscow         summer        right       F         1242
St.Petersburg  summer        right       F         1221
               autumn        right       M         1212
Moscow         autumn        left        M         1192
St.Petersburg  autumn        right       F         1146
Moscow         summer        right       M         1146
               spring        left        M         1131
                             right       F         1108
               summer        left        F         1087
St.Petersburg  summer        left        M         1063


## Пример № 2.2

In [7]:
(df_h.iloc[:,[1,2,3,4]].value_counts(normalize=True)*100).round(3)

city           birth_season  handedness  gender
Moscow         winter        right       F         4.246
                             left        M         4.243
St.Petersburg  spring        right       F         4.235
Moscow         summer        left        M         3.901
               autumn        right       F         3.801
St.Petersburg  winter        left        F         3.732
                                         M         3.592
Moscow         summer        right       F         3.549
St.Petersburg  summer        right       F         3.489
               autumn        right       M         3.463
Moscow         autumn        left        M         3.406
St.Petersburg  autumn        right       F         3.275
Moscow         summer        right       M         3.275
               spring        left        M         3.232
                             right       F         3.166
               summer        left        F         3.106
St.Petersburg  summer        left       

## Пример № 2.3

In [8]:
(df_h.iloc[:,[3,4]].value_counts(normalize=True)*100).round(2)

handedness  gender
right       F         28.40
left        M         26.29
            F         23.36
right       M         21.95
Name: proportion, dtype: float64

# 3. Используем функцию pd.crosstab

## Пример № 3.1

In [9]:
pd.crosstab(index=df_h['handedness'], columns=df_h['gender'])

gender,F,M
handedness,,
left,8176,9199
right,9940,7680


## Пример № 3.2

In [10]:
# другой вариант записи (pd.crosstab(index=df_h['handedness'], columns=df_h['gender'], normalize='all')*100).round(2)

(pd.crosstab(index=df_h['handedness'], columns=df_h['gender'], normalize=True)*100).round(2)

gender,F,M
handedness,,
left,23.36,26.29
right,28.40,21.95


## Пример № 3.3

In [11]:
# рассмотрим другие переменные
display((pd.crosstab(index=df_h['birth_season'], columns=df_h['handedness'], normalize=True)*100).round(2),
        (pd.crosstab(index=df_h['birth_season'], columns=df_h['gender'], normalize=True)*100).round(2))

handedness,left,right
birth_season,,
autumn,11.66,12.93
spring,11.03,12.48
summer,12.80,12.80
winter,14.17,12.14


gender,F,M
birth_season,,
autumn,12.80,11.79
spring,12.85,10.66
summer,12.90,12.70
winter,13.22,13.08


## Пример № 3.4

In [12]:
# Можем добавить итоги All - по строке и по столбцу
pd.crosstab(index=df_h['handedness'], columns=df_h['gender'], margins=True)

gender,F,M,All
handedness,,,
left,8176,9199,17375
right,9940,7680,17620
All,18116,16879,34995


## Пример № 3.5

In [13]:
# то же самое можем сделать с помощью pivot_table, передав в соответвующие параметры нужные аргументы
df_h.pivot_table(index='handedness', columns='gender', values='ID', aggfunc='count', margins=True)

gender,F,M,All
handedness,,,
left,8176,9199,17375
right,9940,7680,17620
All,18116,16879,34995


# 4. Передаем одновременно три переменные (или более)

## Пример № 4.1

In [14]:
# три переменные - 'gender', 'handedness', 'city'
pd.crosstab(index=[df_h['gender'], df_h['handedness']], columns=df_h['city'])

city               Moscow  St.Petersburg
gender handedness                       
F      left          3917           4259
       right         5166           4774
M      left          5173           4026
       right         3674           4006

## Пример № 4.2

In [15]:
# четыре переменные - 'gender', 'handedness', 'city', 'birth_season'
pd.crosstab(index=[df_h['city'], df_h['birth_season']], columns=[df_h['gender'], df_h['handedness']])

gender                         F           M      
handedness                  left right  left right
city          birth_season                        
Moscow        autumn         997  1330  1192   837
              spring         923  1108  1131   875
              summer        1087  1242  1365  1146
              winter         910  1486  1485   816
St.Petersburg autumn        1005  1146   886  1212
              spring         985  1482   820   903
              summer         963  1221  1063   871
              winter        1306   925  1257  1020

# 5. Параметр normalize (index, columns, all (True))

## Пример № 5.1 - normalize='index'

In [16]:
# 100 % по строке - все женщины отдельно и все мужчины отдельно

display(pd.crosstab(index=df_h['gender'], columns=df_h['handedness'], normalize='index').round(3)*100,
        # если подключаем margins=True, то получаем общее количество (или процент) левшей и правшей в последней строчке All, 
        # без разделения по полу
        pd.crosstab(index=df_h['gender'], columns=df_h['handedness'], normalize='index', margins=True).round(3)*100)

handedness,left,right
gender,,
F,45.1,54.9
M,54.5,45.5


handedness,left,right
gender,,
F,45.1,54.9
M,54.5,45.5
All,49.6,50.4


## Пример № 5.2 - normalize='columns'

In [17]:
# 100 % по столбцу - все левши отдельно и все правши отдельно

display(pd.crosstab(index=df_h['gender'], columns=df_h['handedness'], normalize='columns').round(3)*100,
         # если подключаем margins=True, то получаем еще общее количество (или процент) женщин и мужчин в последнем столбце All,
        # без разделения по преобладающей руке
         pd.crosstab(index=df_h['gender'], columns=df_h['handedness'], normalize='columns', margins=True).round(3)*100)

handedness,left,right
gender,,
F,47.1,56.4
M,52.9,43.6


handedness,left,right,All
gender,,,
F,47.1,56.4,51.8
M,52.9,43.6,48.2


## Пример № 5.3 - normalize='all' (normalize=True)

In [18]:
# 100 % - общее количество людей, участвовавших в опросе

display(pd.crosstab(index=df_h['gender'], columns=df_h['handedness'], normalize='all').round(3)*100,
        pd.crosstab(index=df_h['gender'], columns=df_h['handedness'], normalize='all', margins=True).round(3)*100)

handedness,left,right
gender,,
F,23.4,28.4
M,26.3,21.9


handedness,left,right,All
gender,,,
F,23.4,28.4,51.8
M,26.3,21.9,48.2
All,49.6,50.4,100.0


## Пример № 5.4 - более 2-х переменных

In [19]:
# за 100% берется строка (каждая отдельная категория участников - отдельно количество женщин-левшей, женщин-правшей и т.д.)
display(pd.crosstab(index=[df_h['gender'], df_h['handedness']], columns=df_h['city'], normalize='index').round(3)*100)

# за 100% берется столбец (общее количество участников опроса в каждом отдельном городе - в Москве и в Санкт-Петербурге)
display(pd.crosstab(index=[df_h['gender'], df_h['handedness']], columns=df_h['city'], normalize='columns').round(3)*100)

# 100% - все участники опроса
display(pd.crosstab(index=[df_h['gender'], df_h['handedness']], columns=df_h['city'], normalize='all', margins=True)
        .round(3)*100)

city               Moscow  St.Petersburg
gender handedness                       
F      left          47.9           52.1
       right         52.0           48.0
M      left          56.2           43.8
       right         47.8           52.2

city               Moscow  St.Petersburg
gender handedness                       
F      left          21.8           25.0
       right         28.8           28.0
M      left          28.9           23.6
       right         20.5           23.5

city               Moscow  St.Petersburg    All
gender handedness                              
F      left          11.2           12.2   23.4
       right         14.8           13.6   28.4
M      left          14.8           11.5   26.3
       right         10.5           11.4   21.9
All                  51.2           48.8  100.0

# 6. Параметры margins_name, rownames и colnames

## Пример № 6.1

    для изменений используем параметры

    rownames=['sex'], 
    colnames=['hand'], 
    margins_name='all'

In [20]:
# можно изменить названия уровней строк и столбцов результирующей структуры, а также сменить название All с итогами

pd.crosstab(index=df_h['gender'], columns=df_h['handedness'], normalize='all', margins=True, 
            rownames=['sex'], colnames=['hand'], margins_name='all').round(2)*100

hand,left,right,all
sex,,,
F,23.0,28.0,52.0
M,26.0,22.0,48.0
all,50.0,50.0,100.0


## Пример № 6.2

    для изменений используем параметры
    
    rownames=['sex', 'hand'], - для двух уровней индексов строк
    colnames=['CITY'] - для единственного уровня индексов столбцов

In [21]:
pd.crosstab(index=[df_h['gender'], df_h['handedness']], columns=df_h['city'], normalize='all',
            rownames=['sex', 'hand'], colnames=['CITY']).round(2)*100

CITY       Moscow  St.Petersburg
sex hand                        
F   left     11.0           12.0
    right    15.0           14.0
M   left     15.0           12.0
    right    10.0           11.0

# 7. Параметры values и aggfunc в pd.crosstab

    pd.crosstab(index=col1, columns=col2, values=col3, aggfunc=...

    aggfunc= mean / median / sum / min / max - указать одну или несколько агрегирующих функций
    
    aggfunc=['min', 'max']

In [22]:
df_r=pd.read_csv('files/students_results')
df_r.sample(10)

,student_ID,group,subject,points
35,S306,C,French,48
3,S102,A,French,64
19,S204,B,French,83
17,S204,B,English,85
8,S105,A,English,67
33,S306,C,English,56
20,S205,B,English,67
13,S202,B,French,50
22,S205,B,French,37
23,S206,B,French,47


## Пример № 7.1

In [23]:
# вычисляем среднее по группе и по предмету (языку)
pd.crosstab(index=df_r.group, columns=df_r.subject, values=df_r.points, aggfunc='mean').round(2)

subject,English,French
group,,
A,63.83,63.17
B,63.17,61.83
C,63.33,58.83


## Пример № 7.2

In [24]:
# находим самый высокий результат по каждому предмету в каждой отдельной группе
pd.crosstab(index=df_r.group, columns=df_r.subject, values=df_r.points, aggfunc='max')

subject,English,French
group,,
A,86,91
B,85,83
C,81,95


## Пример № 7.3

In [25]:
# находим одновременно самый низкий и самый высокий результат по каждому предмету в каждой отдельной группе
pd.crosstab(df_r.group, df_r.subject, df_r.points, aggfunc=['min', 'max']) 

min            max       
subject English French English French
group                                
A            45     37      86     91
B            34     37      85     83
C            45     45      81     95

# 8. pd.crosstab + Categorical. Параметр dropna

In [26]:
# создадим копию датафрейма, изменив тип данных двух колонок на Categorical, добавив дополнительные категории в конструкторе
df_r2=df_r.copy()
# добавляем лишнюю категорию 'D', по которой данных в ряду нет
df_r2['group']=pd.Categorical(df_r['group'], categories=['A', 'B', 'C','D'])
# добавляем лишнюю категорию 'German', по которой также данных в ряду нет
df_r2['subject']=pd.Categorical(df_r['subject'],categories=['English', 'French', 'German'])
df_r2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   student_ID  36 non-null     object  
 1   group       36 non-null     category
 2   subject     36 non-null     category
 3   points      36 non-null     int64   
dtypes: category(2), int64(1), object(1)
memory usage: 1.1+ KB


## Пример № 8.1

In [27]:
# вычисляем среднее значение
# если dropna=True (по умолчанию)
pd.crosstab(df_r2.group, df_r2.subject, df_r2.points, aggfunc='mean').round(2)

subject,English,French
group,,
A,63.83,63.17
B,63.17,61.83
C,63.33,58.83


## Пример № 8.2

In [28]:
# если dropna=False
pd.crosstab(df_r2.group, df_r2.subject, df_r2.points, aggfunc='mean', dropna=False).round(2) 

subject,English,French,German
group,,,
A,63.83,63.17,NaN
B,63.17,61.83,NaN
C,63.33,58.83,NaN
D,NaN,NaN,NaN
